# 一个MLE的python范例
作者：黄勔 2021年1月2日，2021年4月9日更新

备注：
1. 基于给学生的家庭作业，这里给出一个用python做极大似然估计的例子。叫做范例其实也有不妥，因为我自己才刚刚转到使用python语言做数据分析上来，还有很多东西没有弄明白。叫做learnig by doing可能更好。

作业原题如下：
We are going to use the example on page 13 of logit model. Suppose there are 50 models of cars, the true parameter of $\rho=0.5$, $\theta=-0.3$. The simulated data consists of 1000 consumers. Please write your code to estimate these two parameters and find the variance covariance matrix?

2. 由于自己是从 R 和 Julia 用户转移而来，同时也不清楚 python的 最佳实践方式。借此机会尝试比较一下使用 R 方式和 Julia 方式哪种方式在 python 中更有效。即 R 方式为尽量使用矩阵和向量的运算，避免使用循环，这是 R 和 matlab 这样的矩阵语言比较好的实践方式。Julia 方式为在可以的时候尽量使用基于标量的循环，减少内存的占用，利用 Julia 本身的速度优势。当然 python 由于 numba 的存在，我在两种方式中都尝试用 numba 做JIT（just in time）的实时编译来提高 python 的速度。同时使用 autograd 来实现搜索时的自动微分。(发现无法同时使用 autograd 和 numba，在本例中，我使用 autograd 来提高 gradient 和 hessian 的精度，放弃 jit 加速。如果你知道如何把 autograd 和 numba jit 结合使用，请一定告诉我。)

先引入必要的包

In [7]:
import autograd.numpy as np
import pandas as pd
from autograd import grad, jacobian, hessian
from scipy.optimize import minimize
#from numba import jit 

读入汽车和消费者的数据

In [8]:
data_cars = pd.read_csv("data_cars.csv")
data_cons = pd.read_csv("data_consumers.csv")

In [9]:
print(data_cons)
print(data_cars)

     M          I  choice
0    7  35.065422       3
1    6  31.391821      40
2    2  10.267193      44
3    7  35.081169      40
4    2   9.103030      22
..  ..        ...     ...
995  1   4.156514      28
996  5  23.336381      44
997  4  20.661315      28
998  3  14.506193      20
999  3  16.137090      30

[1000 rows x 3 columns]
    model   SR         PP
0       1  4.3  11.157731
1       2  4.7  13.219156
2       3  5.1  12.270740
3       4  4.3  13.379698
4       5  4.3  11.598031
5       6  4.9  12.604224
6       7  4.0  11.290917
7       8  4.3  12.361849
8       9  4.0  10.615555
9      10  5.1  13.027972
10     11  4.7  12.421642
11     12  4.9  11.769282
12     13  4.7  11.701297
13     14  4.7  12.577615
14     15  4.0  10.392503
15     16  4.0  13.318082
16     17  5.1  13.219362
17     18  4.9  13.360677
18     19  4.9  13.341525
19     20  4.3  13.442825
20     21  4.3  11.336404
21     22  4.7  12.097588
22     23  4.9  14.131405
23     24  4.3  10.902734
24     25  4.

## 使用R语言的方式工作

design matrix for consumers

In [10]:
mat_cons = data_cons[['M', 'I', 'choice']]

In [11]:
mat_cons['1/I'] = 1/mat_cons['I']

In [12]:
mat_cons.drop('I', axis=1, inplace=True)

In [13]:
mat_cons = mat_cons.reindex(columns=['M','1/I','choice'])

In [14]:
mat_cons = np.array(mat_cons)

## design matrix for cars

In [15]:
mat_cars = np.array(data_cars[['SR','PP']])

为个体构造llk

In [16]:
def llk_n_R(data_con_n, theta):
    x_n = data_con_n[:2]
    choice_n = int(data_con_n[2])
    theta=np.array(theta)
    VJ = np.exp((x_n*mat_cars*theta).sum(axis=1))
    Pn = VJ[choice_n-1]/VJ.sum()
    return np.log(Pn)

为数据集构建llk

In [17]:
def neg_llk_R(theta):
    llk = 0
    for row in mat_cons:
        llk += llk_n_R(row, theta)
    return -1*llk

## 开始求极大值

利用autograd生成neg_llk的gradient和hessian以备用

In [18]:
jacobian_  = jacobian(neg_llk_R)
hessian_ = hessian(neg_llk_R)

In [19]:
init_theta = [1,1]

用%timeit来查看求极值需要的时间：

In [23]:
#%timeit res1 = minimize(neg_llk_R, init_theta, method = 'BFGS', options={'disp': False}, jac = jacobian_)
res1 = minimize(neg_llk_R, init_theta, method = 'BFGS', options={'disp': False}, jac = jacobian_)

In [24]:
print(res1)

      fun: 3731.130548040255
 hess_inv: array([[ 0.00111563, -0.00263741],
       [-0.00263741,  0.08937729]])
      jac: array([-1.46790770e-07,  1.01165806e-08])
  message: 'Optimization terminated successfully.'
     nfev: 11
      nit: 8
     njev: 11
   status: 0
  success: True
        x: array([ 0.5374429 , -0.32865209])


得到$\theta$的极大似然估计值：

In [25]:
theta_mle = res1.x

$\theta$ 的渐进方差协方差矩阵为$-\boldsymbol{H}^{-1}/N$，我们用neg_llk在估计值处的$hessian/N$来估计$-\boldsymbol{H}$:

In [26]:
neg_H_inv = np.linalg.inv(hessian_(theta_mle)/mat_cons.shape[0])

In [27]:
var_mat = neg_H_inv/mat_cons.shape[0]

In [28]:
print(var_mat)

[[ 0.00110568 -0.00256991]
 [-0.00256991  0.08891964]]


var_mat对角线元素即$\theta$的极大似然估计的渐进方差